<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/RunnablePassthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. `RunnablePassthrouhg`**
---

Dentro de los Runnables, uno de los más sencillos —pero muy útil— es `RunnablePassthrough`. Este “pasa” sus datos de entrada directamente a la salida sin alterarlos. Puede parecer trivial, pero resulta práctico en situaciones en las que queremos que un eslabón de la cadena no modifique la información que recibe, sirviendo como “puente” para mantener la compatibilidad o facilidad de lectura en la cadena.

# **Preparando el entorno del cuaderno**
---
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google Hugging Face, Mistral, Together y Anthropic

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.
El uso de las API de OpenAI y Anthropic es de pago. El resto son gratuitas y para usarlas basta con registrarse y generar una API Key.  

En el primer cuaderno encontraras los enlaces a estos servicios y este codigo explicado



In [2]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')
MISTRAL_API_KEY=userdata.get('MISTRAL_API_KEY')
TOGETHER_API_KEY=userdata.get('TOGETHER_API_KEY')


# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.


# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU
%pip install langchain_mistralai -qU
%pip install langchain-together -qU
%pip install langchain-anthropic -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint
from langchain_mistralai import ChatMistralAI
from langchain_together import ChatTogether
from langchain_anthropic import ChatAnthropic


# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

## **Ejemplo 1: No hacer "nada"**
---
En su forma más sencilla, `RunnablePassthrough` recibe un input y **devuelve exactamente el mismo output**.

#### **¿Por qué usar algo tan sencillo?**
A veces en una Chain necesitas un paso que no modifique los datos, pero que sea compatible con la secuencia de Runnables. Por ejemplo, un eslabón que valide un formato o simplemente reenvíe la información a otro paso.



In [5]:
from langchain_core.runnables import RunnablePassthrough

# Creamos un RunnablePassthrough
passthrough = RunnablePassthrough()

# Definimos un texto de ejemplo
texto_entrada = "Este texto será pasado sin cambios."

# Ejecutamos el RunnablePassthrough
resultado = passthrough.invoke(texto_entrada)

print("Texto de entrada: ", texto_entrada)
print("Resultado       : ", resultado)


Texto de entrada :  Este texto será pasado sin cambios.
Resultado        :  Este texto será pasado sin cambios.


## **Ejemplo 2: Integrar RunnablePassthrough en una cadena**
---
Este ejempplo puede parecer un poco forzada pero trata de mostrar como se integra facilmente un RunnablePassthrough en una cadena.

Pasaremos un promt a un LLM y el resultado de este pasara a traves del RunnablePassthrough a otro prompt y de ahi a otro LLM.



In [16]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt_template1 = PromptTemplate.from_template("Describe y condensa el siguinte tema en una sola frase: {tema}")
prompt_template2 = PromptTemplate.from_template("Describe y condensa la siguinete frase en una sola palabra: {frase}")

llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

mayusculas = RunnableLambda(lambda x: x.content.upper())

chain = prompt_template1 | llm_gpt4o_mini | RunnablePassthrough() | prompt_template2 | llm_gpt4o_mini | mayusculas

resultado=chain.invoke({"tema": "La inteligencia artificial"})

resultado




'AUTOMATIZACIÓN.'

Te has dado cuenta de la "dificultad" de conocer lo que esta pasando dentro de la cadena ? Cual es la frase generada por el primer modelo ?

Aqui podriamos hacer uso del RunnableLambda para imprimir el resultado intermedio en consola o cualquier otra tarea de debugger


In [29]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt_template1 = PromptTemplate.from_template("Describe y condensa el siguinte tema en una sola frase: {tema}")
prompt_template2 = PromptTemplate.from_template("Describe y condensa la siguinete frase en una sola palabra: {frase}")

llm_gpt4o_mini = ChatOpenAI(model="gpt-4o-mini",api_key=OPENAI_API_KEY, temperature=1)

mayusculas = RunnableLambda(lambda x: x.content.upper())

def imprimir_log(x):
  print(x.content)
  return x  # es importante devolver lo mismo que recibimos

imprimir_log_runnable = RunnableLambda(imprimir_log)

chain = prompt_template1 | llm_gpt4o_mini | imprimir_log_runnable | RunnablePassthrough() | prompt_template2 | llm_gpt4o_mini | mayusculas

resultado=chain.invoke({"tema": "La inteligencia artificial"})

resultado

La inteligencia artificial es la simulación de procesos de inteligencia humana por parte de sistemas computacionales, que buscan realizar tareas como el aprendizaje, razonamiento y autocorrección.


'INTELIGENCIA.'